In [1]:
import pandas as pd 
import numpy as np 

from sklearn.model_selection import train_test_split 

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Classification

In [2]:
data = pd.read_csv('../datasets/dataset_2.csv')
data.shape

(50000, 109)

In [3]:
data.head()

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109
0,4.532710,3.280834,17.982476,4.404259,2.349910,0.603264,2.784655,0.323146,12.009691,0.139346,...,2.079066,6.748819,2.941445,18.360496,17.726613,7.774031,1.473441,1.973832,0.976806,2.541417
1,5.821374,12.098722,13.309151,4.125599,1.045386,1.832035,1.833494,0.709090,8.652883,0.102757,...,2.479789,7.795290,3.557890,17.383378,15.193423,8.263673,1.878108,0.567939,1.018818,1.416433
2,1.938776,7.952752,0.972671,3.459267,1.935782,0.621463,2.338139,0.344948,9.937850,11.691283,...,1.861487,6.130886,3.401064,15.850471,14.620599,6.849776,1.098210,1.959183,1.575493,1.857893
3,6.020690,9.900544,17.869637,4.366715,1.973693,2.026012,2.853025,0.674847,11.816859,0.011151,...,1.340944,7.240058,2.417235,15.194609,13.553772,7.229971,0.835158,2.234482,0.946170,2.700606
4,3.909506,10.576516,0.934191,3.419572,1.871438,3.340811,1.868282,0.439865,13.585620,1.153366,...,2.738095,6.565509,4.341414,15.893832,11.929787,6.954033,1.853364,0.511027,2.599562,0.811364


In [4]:
# separate train and test sets

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((35000, 108), (15000, 108))

## Remove correlated features

Step Forward Feature Selection takes a long time to run, so remove correlated features to reduce the feature space.

In [5]:
def correlation(dataset, threshold):
    col_corr = set() # Set of all the names of correlated columns
    corr_matrix = dataset.corr() 
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                col_corr.add(corr_matrix.columns[i])
            
    return col_corr

corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )


correlated features:  36


In [6]:
# remove correlated features 

X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((35000, 72), (15000, 72))

## Step Forward Feature Selection

documentation: http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/

In [7]:
sfs = SFS(RandomForestClassifier(n_estimators=10, n_jobs=4, random_state=0), 
           k_features=10, # the more features we want, the longer it will take to run
           forward=False, 
           floating=False, # see the docs for more details in this parameter
           verbose=2, # this indicates how much to print out intermediate steps
           scoring='roc_auc',
           cv=2)

sfs = sfs.fit(np.array(X_train), y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:  2.1min finished

[2021-05-04 17:00:47] Features: 71/10 -- score: 0.6268872842282114[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  71 out of  71 | elapsed:  2.0min finished

[2021-05-04 17:02:44] Features: 70/10 -- score: 0.6307351898704118[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:  1.9min finished

[2021-05-04 17:04:36] Features: 69/10 -- score: 0.6300809921574941[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Don

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   58.1s finished

[2021-05-04 17:39:08] Features: 44/10 -- score: 0.634992341221544[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  44 out of  44 | elapsed:   56.8s finished

[2021-05-04 17:40:05] Features: 43/10 -- score: 0.6355070764261569[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  43 out of  43 | elapsed:   55.9s finished

[2021-05-04 17:41:01] Features: 42/10 -- score: 0.6365981238453224[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  42 out of  42 | elapsed:   53.3s finished

[2021-05-04 17:41:54] Features: 41/1

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:   16.7s finished

[2021-05-04 17:55:41] Features: 16/10 -- score: 0.6449535484605471[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:   14.2s finished

[2021-05-04 17:55:55] Features: 15/10 -- score: 0.6426139092439326[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   10.1s finished

[2021-05-04 17:56:05] Features: 14/10 -- score: 0.6433088715464276[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done

In [8]:
selected_features = X_train.columns[list(sfs.k_feature_idx_)]
selected_features

Index(['var_16', 'var_21', 'var_30', 'var_45', 'var_51', 'var_55', 'var_62',
       'var_69', 'var_91', 'var_96'],
      dtype='object')

## Compare performance of feature subsets

In [9]:
def run_random_forest(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=200, random_state=0, max_depth=4)
    clf.fit(X_train, y_train)
    
    print('Train set')
    pred = clf.predict_proba(X_train)
    print(f"roc-auc: {roc_auc_score(y_train, pred[:, 1])}")
    
    print('Test set')
    pred = clf.predict_proba(X_test)
    print(f"roc-auc: {roc_auc_score(y_test, pred[:, 1])}")

In [10]:
# without any feature selection
run_random_forest(X_train, X_test, y_train, y_test)

Train set
roc-auc: 0.7121671720188072
Test set
roc-auc: 0.6955081921832347


In [11]:
# with feature selection
run_random_forest(X_train[selected_features], X_test[selected_features], y_train, y_test)

Train set
roc-auc: 0.7121967258531186
Test set
roc-auc: 0.7016768503675828


Random forest with all the features (72) and only 10 selected features have pretty similar performance.

# Regression

In [12]:
# load data 
data = pd.read_csv('../datasets/houseprice.csv')
data.shape

(1460, 81)

In [13]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [14]:
# sparate train and test sets 

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0
)

X_train.shape, X_test.shape

((1022, 37), (438, 37))

## Remove correlated features 

In [15]:
corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  3


In [16]:
# removed correlated features
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((1022, 34), (438, 34))

In [17]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

## Step Forward Feature Selection

In [18]:
sfs = SFS(
    RandomForestRegressor(n_estimators=10, n_jobs=4, random_state=0),
    k_features=20,
    forward=False,
    floating=False,
    verbose=2,
    scoring='r2',
    cv=2
)

sfs = sfs.fit(np.array(X_train), y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:   10.2s finished

[2021-05-04 17:57:11] Features: 33/20 -- score: 0.8277321480525341[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    9.9s finished

[2021-05-04 17:57:21] Features: 32/20 -- score: 0.8262089705934443[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    9.5s finished

[2021-05-04 17:57:31] Features: 31/20 -- score: 0.8268300137600382[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Don

In [19]:
# selected columns
selected_feat = X_train.columns[list(sfs.k_feature_idx_)]
selected_feat

Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces',
       'GarageCars', 'EnclosedPorch', '3SsnPorch', 'PoolArea', 'MoSold'],
      dtype='object')

## Compare performance of feature subsets

In [20]:
# function to train random forests and evaluate the performance

def run_randomForests(X_train, X_test, y_train, y_test):
    
    rf = RandomForestRegressor(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)

    print('Train set')
    pred = rf.predict(X_train)
    print('Random Forests roc-auc: {}'.format(r2_score(y_train, pred)))
    
    print('Test set')
    pred = rf.predict(X_test)
    print('Random Forests roc-auc: {}'.format(r2_score(y_test, pred)))

In [21]:
# without any feature selection
run_randomForests(X_train, X_test, y_train, y_test)

Train set
Random Forests roc-auc: 0.8699152317492538
Test set
Random Forests roc-auc: 0.8190809813112794


In [22]:
# with feature selection
run_randomForests(X_train[selected_feat], X_test[selected_feat], y_train, y_test)

Train set
Random Forests roc-auc: 0.8640077322487285
Test set
Random Forests roc-auc: 0.8142081879236611


The algorithm with 20 features performs as well as that with 24 features.